<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Use-caution.--Not-all-examples-will-work!" data-toc-modified-id="Use-caution.--Not-all-examples-will-work!-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Use caution.  Not all examples will work!</a></span></li></ul></div>

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

### Upload radiation.txt to your Google Drive NLP_data folder

In [7]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
 
def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
    article = filedata[0].split(". ")
    sentences = []

    for sentence in article:
        print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop() 
    
    return sentences

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix


def generate_summary(file_name, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences =  read_article(file_name)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarize texr
    print("Summarize Text: \n", ". ".join(summarize_text))

# let's begin
generate_summary('/Users/jimcody/Documents/2021Python/nlp/data/radiation.txt', 2)

Radiation is used in healthcare procedures to help providers find causes of symptoms (diagnostics) and to manage or treat health conditions
Although we all are exposed to ionizing radiation every day, any added exposures, including from imaging procedures, slightly increases the risk of developing cancer later in life
Usually, the benefits of diagnosing or treating a health problem with an imaging procedure will outweigh these risks
Learn about the risks and benefits of common medical imaging procedures
Talk to your healthcare provider about the specific risks and benefits of a recommended test for your situation and how to limit your exposure to radiation.}
Indexes of top ranked_sentence order are  [(0.34165391554554825, ['Learn', 'about', 'the', 'risks', 'and', 'benefits', 'of', 'common', 'medical', 'imaging', 'procedures']), (0.32275842815351563, ['Usually,', 'the', 'benefits', 'of', 'diagnosing', 'or', 'treating', 'a', 'health', 'problem', 'with', 'an', 'imaging', 'procedure', 'wil

### Use caution.  Not all examples will work!

Gensim removed gensim.summarization because 

In [ ]:
!pip install gensim.summarization
import gensim
from gensim.summarization import summarize

In [ ]:
# WILL NOT WORK!!!

from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
import wikipedia
import en_core_web_sm
 
# Get wiki content.
wikisearch = wikipedia.page("Amitabh Bachchan")
wikicontent = wikisearch.content
nlp = en_core_web_sm.load()
doc = nlp(wikicontent)
 
# Save the wiki content to a file
# (for reference).
f = open("wikicontent.txt", "w")
f.write(wikicontent)
f.close()
 
# Summary (0.5% of the original content).
summ_per = summarize(wikicontent, ratio = 0.05)
print("Percent summary")
print(summ_per)
 
# Summary (200 words)
summ_words = summarize(wikicontent, word_count = 200)
print("Word count summary")
print(summ_words)